In [202]:
import numpy as np
import pandas as pd

In [203]:
df = pd.read_csv('Dataset/train.csv')

In [204]:
columns = ['damage_grade']
y = df.damage_grade
y = pd.get_dummies(y, dummy_na=False)
print(y[:10])
columns = ['has_geotechnical_risk_fault_crack',
          'has_geotechnical_risk_flood',
          'has_geotechnical_risk_land_settlement',
          'has_geotechnical_risk_landslide',
          'has_geotechnical_risk_liquefaction',
          'has_geotechnical_risk_other',
          'has_geotechnical_risk_rock_fall'
          ]
x = pd.DataFrame(df, columns=columns)

column_has_repair_started = []
for i in df.has_repair_started:
    if i == 0.0 or i == 1.0:
        column_has_repair_started.append(int(i))
    else:
        column_has_repair_started.append(0)
print(column_has_repair_started[:10])
has_repair_started = pd.DataFrame(column_has_repair_started)
x = pd.concat([x, has_repair_started],axis=1) 
x = x.rename(columns={0: 'has_repair_started'})

column_has_geotechnical_risk = []
for i in df.has_geotechnical_risk:
    if i == 0.0 or i == 1.0:
        column_has_geotechnical_risk.append(int(i))
    else:
        column_has_geotechnical_risk.append(0)
print(column_has_geotechnical_risk[:10])
has_geotechnical_risk = pd.DataFrame(column_has_geotechnical_risk)
x = pd.concat([x, has_geotechnical_risk],axis=1) 
x = x.rename(columns={0: 'has_geotechnical_risk'})

has_geotechnical_risk = df.has_geotechnical_risk
has_geotechnical_risk = pd.get_dummies(has_geotechnical_risk, dummy_na=False)
print(has_geotechnical_risk[:10])
x = pd.concat([has_geotechnical_risk, x],axis=1)

district_id = df.district_id
district_id = pd.get_dummies(district_id, dummy_na=False)
print(district_id[:10])
x = pd.concat([district_id, x],axis=1) 

   Grade 1  Grade 2  Grade 3  Grade 4  Grade 5
0        0        0        0        1        0
1        0        1        0        0        0
2        1        0        0        0        0
3        0        0        0        0        1
4        0        0        1        0        0
5        0        0        0        0        1
6        0        1        0        0        0
7        0        0        0        0        1
8        1        0        0        0        0
9        0        0        1        0        0
[0, 1, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
   0.0  1.0
0    1    0
1    1    0
2    1    0
3    1    0
4    1    0
5    1    0
6    0    1
7    1    0
8    1    0
9    1    0
   7   9   10  11  12  13  20  21  22  23 ...  38  39  40  43  44  45  46  47  \
0   0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   0   0   0   
1   0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   1   0   0   0   
2   0   0   0   0   0   0   0   0   0   0 ... 

In [205]:
print(df.dtypes)

area_assesed                              object
building_id                               object
damage_grade                              object
district_id                                int64
has_geotechnical_risk                    float64
has_geotechnical_risk_fault_crack          int64
has_geotechnical_risk_flood                int64
has_geotechnical_risk_land_settlement      int64
has_geotechnical_risk_landslide            int64
has_geotechnical_risk_liquefaction         int64
has_geotechnical_risk_other                int64
has_geotechnical_risk_rock_fall            int64
has_repair_started                       float64
vdcmun_id                                  int64
dtype: object


In [206]:
print(x.shape)

(631761, 42)


In [256]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LeakyReLU

In [258]:
model = Sequential()
model.add(Dense(44, input_dim=42, activation='linear'))
model.add(Dense(29, activation='linear'))
model.add(Dropout(0.2))
model.add(Dense(17, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 44)                1892      
_________________________________________________________________
dense_22 (Dense)             (None, 29)                1305      
_________________________________________________________________
dropout_7 (Dropout)          (None, 29)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 17)                510       
_________________________________________________________________
dense_24 (Dense)             (None, 10)                180       
_________________________________________________________________
dropout_8 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_25 (Dense)             (None, 5)                 55        
Total para

In [259]:
train_data = x[:500000]
train_labels = y[:500000]
test_data = x[500000:]
test_labels = y[500000:]

In [260]:
model.fit(train_data, train_labels, validation_data=(test_data, test_labels), epochs=100, batch_size=10000)
scores = model.evaluate(test_data, test_labels, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
model.save('my_model.h5')

Train on 500000 samples, validate on 131761 samples
Epoch 1/100
500000/500000 [==============================] - 2s 5us/step - loss: 1.5372 - acc: 0.2980 - val_loss: 1.4276 - val_acc: 0.3755
Epoch 2/100
500000/500000 [==============================] - 2s 4us/step - loss: 1.4060 - acc: 0.3706 - val_loss: 1.3567 - val_acc: 0.3956
Epoch 3/100
500000/500000 [==============================] - 2s 4us/step - loss: 1.3715 - acc: 0.3966 - val_loss: 1.3415 - val_acc: 0.4247
Epoch 4/100
500000/500000 [==============================] - 2s 4us/step - loss: 1.3595 - acc: 0.4053 - val_loss: 1.3355 - val_acc: 0.4257
Epoch 5/100
500000/500000 [==============================] - 3s 5us/step - loss: 1.3533 - acc: 0.4144 - val_loss: 1.3326 - val_acc: 0.4259
Epoch 6/100
500000/500000 [==============================] - 3s 5us/step - loss: 1.3502 - acc: 0.4185 - val_loss: 1.3303 - val_acc: 0.4288
Epoch 7/100
500000/500000 [==============================] - 2s 4us/step - loss: 1.3478 - acc: 0.4198 - val_loss: 

In [261]:
df = pd.read_csv('Dataset/test.csv')
print(df.shape)
columns = ['has_geotechnical_risk_fault_crack',
          'has_geotechnical_risk_flood',
          'has_geotechnical_risk_land_settlement',
          'has_geotechnical_risk_landslide',
          'has_geotechnical_risk_liquefaction',
          'has_geotechnical_risk_other',
          'has_geotechnical_risk_rock_fall'
          ]
xnew = pd.DataFrame(df, columns=columns)

column_has_repair_started = []
for i in df.has_repair_started:
    if i == 0.0 or i == 1.0:
        column_has_repair_started.append(int(i))
    else:
        column_has_repair_started.append(0)
has_repair_started = pd.DataFrame(column_has_repair_started)
xnew = pd.concat([xnew, has_repair_started],axis=1) 
xnew = xnew.rename(columns={0: 'has_repair_started'})

column_has_geotechnical_risk = []
for i in df.has_geotechnical_risk:
    if i == 0.0 or i == 1.0:
        column_has_geotechnical_risk.append(int(i))
    else:
        column_has_geotechnical_risk.append(0)
has_geotechnical_risk = pd.DataFrame(column_has_geotechnical_risk)
xnew = pd.concat([has_geotechnical_risk, xnew],axis=1) 
xnew = xnew.rename(columns={0: 'has_geotechnical_risk'})

has_geotechnical_risk = df.has_geotechnical_risk
has_geotechnical_risk = pd.get_dummies(has_geotechnical_risk, dummy_na=False)
xnew = pd.concat([has_geotechnical_risk, xnew],axis=1)

district_id = df.district_id
district_id = pd.get_dummies(district_id, dummy_na=False)
xnew = pd.concat([district_id, xnew],axis=1) 

(421175, 13)


In [264]:
ynew = model.predict(xnew)

In [265]:
grade = []
for i in ynew:
    ind = np.unravel_index(np.argmax(i, axis=None), i.shape)
    if ind == (0, ):
        grade.append('Grade 1')
    elif ind == (1, ):
        grade.append('Grade 2')
    elif ind == (2, ):
        grade.append('Grade 3')
    elif ind == (3, ):
        grade.append('Grade 4')
    elif ind == (4, ):
        grade.append('Grade 5')

In [266]:
print(len(grade))

421175


In [267]:
building_id = df.building_id
grade_id = pd.DataFrame(grade)
result = pd.concat([building_id, grade_id],axis=1) 

In [268]:
result = result.rename(columns={0: 'damage_grade'})
print(result[:10])
result.to_csv('submission.csv',mode = 'w', index=False)

   building_id damage_grade
0   a3380c4f75      Grade 5
1   a338a4e653      Grade 5
2   a338a4e6b7      Grade 5
3   a33a6eaa3a      Grade 5
4   a33b073ff6      Grade 5
5  6604e4896c6      Grade 5
6   a33b07430f      Grade 5
7   a33c386cf3      Grade 5
8   a33c386ee7      Grade 5
9   a33c38700f      Grade 5


In [348]:
Building_Ownership_Use = pd.read_csv('Dataset/Building_Ownership_Use.csv')
Building_Structure = pd.read_csv('Dataset/Building_Structure.csv')
Building_Structure.drop(Building_Structure.columns[0], axis=1, inplace=True)
record = pd.concat([Building_Ownership_Use, Building_Structure],axis=1)
masterReco = record
land_surface_condition = pd.get_dummies(masterReco.land_surface_condition, dummy_na=False)
foundation_type = pd.get_dummies(masterReco.foundation_type, dummy_na=False)
roof_type = pd.get_dummies(masterReco.roof_type, dummy_na=False)
ground_floor_type = pd.get_dummies(masterReco.ground_floor_type, dummy_na=False)
other_floor_type = pd.get_dummies(masterReco.other_floor_type, dummy_na=False)
position = pd.get_dummies(masterReco.position, dummy_na=False)
plan_configuration = pd.get_dummies(masterReco.plan_configuration, dummy_na=False)
legal_ownership_status = pd.get_dummies(masterReco.legal_ownership_status, dummy_na=False)
condition_post_eq = pd.get_dummies(masterReco.condition_post_eq, dummy_na=False)
col = [
    land_surface_condition,
    foundation_type,
    roof_type,
    ground_floor_type,
    other_floor_type,
    position,
    plan_configuration,
    legal_ownership_status,
    condition_post_eq
      ]
m = pd.concat(col,axis=1) 
building_id = masterReco.building_id
columns_drop = [
    "land_surface_condition",
    "foundation_type",
    "roof_type",
    "ground_floor_type",
    "other_floor_type",
    "position",
    "plan_configuration",
    "legal_ownership_status",
    "condition_post_eq",
    "district_id",
    "vdcmun_id",
    "ward_id",
    "building_id"
      ]
masterReco.drop(columns_drop, axis=1, inplace=True)
masterReco = pd.concat([masterReco, m],axis=1) 
masterReco = masterReco.astype(float)
masterReco = pd.concat([building_id, masterReco],axis=1)
masterReco.set_index("building_id", inplace=True)
print(masterReco.dtypes)

count_families                             float64
has_secondary_use                          float64
has_secondary_use_agriculture              float64
has_secondary_use_hotel                    float64
has_secondary_use_rental                   float64
has_secondary_use_institution              float64
has_secondary_use_school                   float64
has_secondary_use_industry                 float64
has_secondary_use_health_post              float64
has_secondary_use_gov_office               float64
has_secondary_use_use_police               float64
has_secondary_use_other                    float64
count_floors_pre_eq                        float64
count_floors_post_eq                       float64
age_building                               float64
plinth_area_sq_ft                          float64
height_ft_pre_eq                           float64
height_ft_post_eq                          float64
has_superstructure_adobe_mud               float64
has_superstructure_mud_mortar_s

In [466]:
df = pd.read_csv('Dataset/train.csv')
flag = masterReco.loc[[df.building_id[0]]]
for i in range(1, len(df)):
    temp = masterReco.loc[[df.building_id[i]]]
    flag = pd.concat([flag,temp],axis=0)
    if i%10000==0:
        print(int(i*100/len(df)), '% records')
print('Completed finding all the results')
flag = flag.reset_index()

1 % records
3 % records
4 % records
6 % records
7 % records
9 % records
11 % records
12 % records
14 % records
15 % records
17 % records
18 % records
20 % records
22 % records
23 % records
25 % records
26 % records
28 % records
30 % records
31 % records
33 % records
34 % records
36 % records
37 % records
39 % records
41 % records
42 % records
44 % records
45 % records
47 % records
49 % records
50 % records
52 % records
53 % records
55 % records
56 % records
58 % records
60 % records
61 % records
63 % records
64 % records
66 % records
68 % records
71 % records
72 % records
74 % records
75 % records
77 % records
79 % records
80 % records
82 % records
83 % records
85 % records
87 % records
88 % records
90 % records
91 % records
93 % records
94 % records
96 % records
98 % records
99 % records
Completed finding all the results


In [474]:
saved_record = pd.concat([df, flag], axis=1)
saved_record.to_csv('advanced_train_set.csv')

In [479]:
print(saved_record.head(5))

       area_assesed  building_id damage_grade  district_id  \
0              Both  24385bfd2a2      Grade 4           24   
1              Both  405d1bbebbf      Grade 2           44   
2              Both  351d9bc71f6      Grade 1           36   
3  Building removed  2be3a971166      Grade 5           30   
4              Both  34c7d073ea6      Grade 3           36   

   has_geotechnical_risk  has_geotechnical_risk_fault_crack  \
0                    0.0                                  0   
1                    0.0                                  0   
2                    0.0                                  0   
3                    0.0                                  0   
4                    0.0                                  0   

   has_geotechnical_risk_flood  has_geotechnical_risk_land_settlement  \
0                            0                                      0   
1                            0                                      0   
2                            

In [565]:
df = pd.read_csv('advanced_train_set.csv')
columns = ['damage_grade']
y = df.damage_grade
y = pd.get_dummies(y, dummy_na=False)

district_id = df.district_id
district_id = pd.get_dummies(district_id, dummy_na=False)
df = pd.concat([df, district_id],axis=1)
area_assesed = df.area_assesed
area_assesed = pd.get_dummies(area_assesed, dummy_na=False)
df = pd.concat([df, area_assesed],axis=1)
df = df.drop(['building_id', 'damage_grade', 'building_id.1', 'district_id', 'area_assesed', 'vdcmun_id', 'Unnamed: 0'], axis = 1)
df = df.astype(float)
df = df.fillna(0)
x = df

In [566]:
print(x.dtypes)

has_geotechnical_risk                    float64
has_geotechnical_risk_fault_crack        float64
has_geotechnical_risk_flood              float64
has_geotechnical_risk_land_settlement    float64
has_geotechnical_risk_landslide          float64
has_geotechnical_risk_liquefaction       float64
has_geotechnical_risk_other              float64
has_geotechnical_risk_rock_fall          float64
has_repair_started                       float64
count_families                           float64
has_secondary_use                        float64
has_secondary_use_agriculture            float64
has_secondary_use_hotel                  float64
has_secondary_use_rental                 float64
has_secondary_use_institution            float64
has_secondary_use_school                 float64
has_secondary_use_industry               float64
has_secondary_use_health_post            float64
has_secondary_use_gov_office             float64
has_secondary_use_use_police             float64
has_secondary_use_ot

In [594]:
import keras
model = Sequential()
model.add(Dense(120, input_dim=120, activation='relu'))
model.add(Dense(70, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(40, activation='sigmoid'))
model.add(Dropout(0.6))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_105 (Dense)            (None, 120)               14520     
_________________________________________________________________
dense_106 (Dense)            (None, 70)                8470      
_________________________________________________________________
dense_107 (Dense)            (None, 30)                2130      
_________________________________________________________________
dense_108 (Dense)            (None, 40)                1240      
_________________________________________________________________
dropout_28 (Dropout)         (None, 40)                0         
_________________________________________________________________
dense_109 (Dense)            (None, 5)                 205       
Total params: 26,565
Trainable params: 26,565
Non-trainable params: 0
_________________________________________________________________
None


In [595]:
train_data = x[:500000]
train_labels = y[:500000]
test_data = x[500000:]
test_labels = y[500000:]
print(train_data[:5])

   has_geotechnical_risk  has_geotechnical_risk_fault_crack  \
0                    0.0                                0.0   
1                    0.0                                0.0   
2                    0.0                                0.0   
3                    0.0                                0.0   
4                    0.0                                0.0   

   has_geotechnical_risk_flood  has_geotechnical_risk_land_settlement  \
0                          0.0                                    0.0   
1                          0.0                                    0.0   
2                          0.0                                    0.0   
3                          0.0                                    0.0   
4                          0.0                                    0.0   

   has_geotechnical_risk_landslide  has_geotechnical_risk_liquefaction  \
0                              0.0                                 0.0   
1                              0.0

In [596]:
model.fit(train_data, train_labels, validation_data=(test_data, test_labels), epochs=10, batch_size=100)
scores = model.evaluate(test_data, test_labels, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
model.save('advanced_trained_model.h5')

Train on 500000 samples, validate on 131761 samples
Epoch 1/10
500000/500000 [==============================] - 26s 51us/step - loss: 0.7956 - acc: 0.6533 - val_loss: 0.6524 - val_acc: 0.7082
Epoch 2/10
500000/500000 [==============================] - 25s 49us/step - loss: 0.6799 - acc: 0.6972 - val_loss: 0.6379 - val_acc: 0.7097
Epoch 3/10
500000/500000 [==============================] - 26s 52us/step - loss: 0.6660 - acc: 0.7012 - val_loss: 0.6344 - val_acc: 0.7124
Epoch 4/10
500000/500000 [==============================] - 29s 57us/step - loss: 0.6573 - acc: 0.7042 - val_loss: 0.6275 - val_acc: 0.7153
Epoch 5/10
500000/500000 [==============================] - 28s 56us/step - loss: 0.6522 - acc: 0.7062 - val_loss: 0.6322 - val_acc: 0.7128
Epoch 6/10
500000/500000 [==============================] - 29s 58us/step - loss: 0.6469 - acc: 0.7087 - val_loss: 0.6295 - val_acc: 0.7120
Epoch 7/10
500000/500000 [==============================] - 25s 51us/step - loss: 0.6456 - acc: 0.7093 - val

In [ ]:
df_test = pd.read_csv('Dataset/test.csv')
flag_test = masterReco.loc[[df_test.building_id[0]]]
for i in range(1, len(df_test)):
    temp_test = masterReco.loc[[df_test.building_id[i]]]
    flag_test = pd.concat([flag_test,temp_test],axis=0)
    if i%10000==0:
        print(int(i*100/len(df_test)), '% records')
print('Completed finding all the results')
flag_test = flag.reset_index()
saved_record_test = pd.concat([df_test, flag_test], axis=1)
saved_record_test.to_csv('advanced_test_set.csv')

2 % records
4 % records
7 % records
9 % records
11 % records
14 % records
16 % records
18 % records
21 % records
23 % records
26 % records
28 % records
30 % records
33 % records
35 % records
37 % records
40 % records
42 % records
45 % records
47 % records
49 % records
52 % records
54 % records


In [ ]:
df_test = pd.read_csv('advanced_test_set.csv')
columns = ['damage_grade']
district_id = df_test.district_id
district_id = pd.get_dummies(district_id, dummy_na=False)
df_test = pd.concat([df_test, district_id],axis=1)
area_assesed = df_test.area_assesed
area_assesed = df_test.get_dummies(area_assesed, dummy_na=False)
df_test = pd.concat([df_test, area_assesed],axis=1)
df_test = df_test.drop(['building_id', 'damage_grade', 'building_id.1', 'district_id', 'area_assesed', 'vdcmun_id', 'Unnamed: 0'], axis = 1)
df_test = df_test.astype(float)
df_test = df_test.fillna(0)
xnew_test = df_test

In [ ]:
from keras.models import load_model
model = load_model('advanced_trained_model.best.h5')
ynew_test = model.predict(xnew_test)
grade = []
for i in ynew:
    ind = np.unravel_index(np.argmax(i, axis=None), i.shape)
    if ind == (0, ):
        grade.append('Grade 1')
    elif ind == (1, ):
        grade.append('Grade 2')
    elif ind == (2, ):
        grade.append('Grade 3')
    elif ind == (3, ):
        grade.append('Grade 4')
    elif ind == (4, ):
        grade.append('Grade 5')

In [ ]:
print(len(grade))
building_id = df.building_id
grade_id = pd.DataFrame(grade)
result = pd.concat([building_id, grade_id],axis=1) 
result = result.rename(columns={0: 'damage_grade'})
print(result[:10])
result.to_csv('submission_advanced.csv',mode = 'w', index=False)